In [3]:
import numpy as np
print(np.random.permutation(10))

[2 8 7 1 3 4 9 0 6 5]


In [1]:
from skimage.metrics import structural_similarity

In [5]:
train = 40000
dev = 1000
test = 1000

img_list = np.random.permutation(train+dev+test)

In [8]:
i = (1, 3, 5)
i[0]

1

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision.models as tmodels
from functools import partial
import collections

# dummy data: 10 batches of images with batch size 16
dataset = [torch.rand(16,3,224,224).cuda() for _ in range(10)]

# network: a resnet50
net = tmodels.resnet50(pretrained=True).cuda()

# a dictionary that keeps saving the activations as they come
activations = collections.defaultdict(list)
def save_activation(name, mod, inp, out):
	activations[name].append(out.cpu())

# Registering hooks for all the Conv2d layers
# Note: Hooks are called EVERY TIME the module performs a forward pass. For modules that are
# called repeatedly at different stages of the forward pass (like RELUs), this will save different
# activations. Editing the forward pass code to save activations is the way to go for these cases.
for name, m in net.named_modules():
	if type(m)==nn.Conv2d:
		# partial to assign the layer name to each hook
		m.register_forward_hook(partial(save_activation, name))

# forward pass through the full dataset
for batch in dataset:
	out = net(batch)

# concatenate all the outputs we saved to get the the activations for each layer for the whole dataset
activations = {name: torch.cat(outputs, 0) for name, outputs in activations.items()}

# just print out the sizes of the saved activations as a sanity check
for k,v in activations.items():
	print (k, v.size())

In [16]:
def mse_froim_psnr(psnr):
    mse = 10**-(psnr/20)
    return mse

In [17]:
print(mse_froim_psnr(25.030))

0.05604024173641845


In [10]:
def percent_improv(a, b):
    return 100*((a-b)/b)

In [18]:
print(percent_improv(0.085, 0.056))

51.78571428571429


In [36]:
import numpy as np
import matplotlib.pyplot as plt

def plot_dataset_vs_time():

    # Create some mock data
    dset= ['1k', '10k', '20k', '40k', '50k']
    data1 = [25.15, 26.83, 26.98, 27.28, 27.31]
    data2 = [2.55, 21.10, 41.90, 83.78, 104.68]

    fig, ax1 = plt.subplots()

    color = 'purple'
    ax1.set_xlabel('Dataset Size')
    ax1.set_ylabel('PSNR (dB)', color=color)
    ax1.plot(dset, data1, color=color, marker='o', ls='--')
    ax1.plot('40k', 27.28, 'ko')
    ax1.tick_params(axis='y', labelcolor=color)
    ax1.yaxis.label.set_size(16)

    ax1.xaxis.label.set_size(16)

    ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

    color = 'green'
    ax2.set_ylabel('Training Time (s)', color=color)  # we already handled the x-label with ax1
    ax2.plot(dset, data2, color=color, marker='o', ls='--')
    ax2.plot('40k', 83.27, 'ko')
    ax2.tick_params(axis='y', labelcolor=color)
    ax2.yaxis.label.set_size(16)

    fig.tight_layout()  # otherwise the right y-label is slightly clipped
    # plt.show()
    plt.savefig('datasetVtime.eps', dpi=600)

In [37]:
plot_dataset_vs_time()

ValueError: Unrecognized character 0 in format string